# Домашнее задание к уроку 3: Полносвязные сети

Задание 1: Эксперименты с глубиной сети (30 баллов)

Для экспериментов буду использовать датасет Титаник из прошлого урока

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
import os
import torch.nn as nn

class ClassificationMetrics:
    def __init__(self):
        """
        Инициализация класса метрик классификации
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def to_one_hot(self, labels, num_classes):
        """
        Преобразование меток в one-hot encoding
        
        Args:
            labels: тензор меток (batch_size,)
            num_classes: количество классов
            
        Returns:
            one-hot тензор (batch_size, num_classes)
        """
        return torch.nn.functional.one_hot(labels, num_classes).float()

    def precision(self, y_true, y_pred, average='macro'):
        """
        Вычисление метрики Precision
        
        Args:
            y_true: истинные метки (batch_size,)
            y_pred: предсказанные метки (batch_size,)
            average: тип усреднения ('macro', 'micro', 'weighted')
            
        Returns:
            precision score
        """
        y_true = torch.tensor(y_true, dtype=torch.long).to(self.device)
        y_pred = torch.tensor(y_pred, dtype=torch.long).to(self.device)
        
        num_classes = len(torch.unique(y_true))
        
        if average == 'micro':
            true_positives = torch.sum(y_true == y_pred).float()
            predicted_positives = len(y_pred)
            return true_positives / predicted_positives if predicted_positives > 0 else 0
        
        precision_per_class = []
        weights = []
        
        for c in range(num_classes):
            true_positives = torch.sum((y_true == c) & (y_pred == c)).float()
            predicted_positives = torch.sum(y_pred == c).float()
            
            precision = true_positives / predicted_positives if predicted_positives > 0 else 0
            precision_per_class.append(precision)
            
            if average == 'weighted':
                weights.append(torch.sum(y_true == c).float() / len(y_true))
        
        precision_per_class = torch.tensor(precision_per_class)
        
        if average == 'macro':
            return torch.mean(precision_per_class).item()
        elif average == 'weighted':
            return torch.sum(precision_per_class * torch.tensor(weights)).item()
        
        return precision_per_class.tolist()

    def recall(self, y_true, y_pred, average='macro'):
        """
        Вычисление метрики Recall
        
        Args:
            y_true: истинные метки (batch_size,)
            y_pred: предсказанные метки (batch_size,)
            average: тип усреднения ('macro', 'micro', 'weighted')
            
        Returns:
            recall score
        """
        y_true = torch.tensor(y_true, dtype=torch.long).to(self.device)
        y_pred = torch.tensor(y_pred, dtype=torch.long).to(self.device)
        
        num_classes = len(torch.unique(y_true))
        
        if average == 'micro':
            true_positives = torch.sum(y_true == y_pred).float()
            actual_positives = len(y_true)
            return true_positives / actual_positives if actual_positives > 0 else 0
        
        recall_per_class = []
        weights = []
        
        for c in range(num_classes):
            true_positives = torch.sum((y_true == c) & (y_pred == c)).float()
            actual_positives = torch.sum(y_true == c).float()
            
            recall = true_positives / actual_positives if actual_positives > 0 else 0
            recall_per_class.append(recall)
            
            if average == 'weighted':
                weights.append(torch.sum(y_true == c).float() / len(y_true))
        
        recall_per_class = torch.tensor(recall_per_class)
        
        if average == 'macro':
            return torch.mean(recall_per_class).item()
        elif average == 'weighted':
            return torch.sum(recall_per_class * torch.tensor(weights)).item()
        
        return recall_per_class.tolist()

    def f1_score(self, y_true, y_pred, average='macro'):
        """
        Вычисление метрики F1-score
        
        Args:
            y_true: истинные метки (batch_size,)
            y_pred: предсказанные метки (batch_size,)
            average: тип усреднения ('macro', 'micro', 'weighted')
            
        Returns:
            f1 score
        """
        precision = self.precision(y_true, y_pred, average)
        recall = self.recall(y_true, y_pred, average)
        
        if average in ['macro', 'weighted']:
            return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        elif average == 'micro':
            return precision
        
        f1_scores = []
        for p, r in zip(precision, recall):
            f1 = 2 * (p * r) / (p + r) if (p + r) > 0 else 0
            f1_scores.append(f1)
        return f1_scores

    def roc_auc(self, y_true, y_scores, multi_class='ovr'):
        """
        Вычисление метрики ROC-AUC
        
        Args:
            y_true: истинные метки (batch_size,)
            y_scores: вероятности предсказаний (batch_size, num_classes)
            multi_class: 'ovr' (one-vs-rest) или 'ovo' (one-vs-one)
            
        Returns:
            roc-auc score
        """
        y_true = torch.tensor(y_true, dtype=torch.long).cpu().numpy()
        y_scores = torch.tensor(y_scores, dtype=torch.float).cpu().numpy()
        
        num_classes = y_scores.shape[1]
        y_true_one_hot = self.to_one_hot(torch.tensor(y_true), num_classes).cpu().numpy()
        
        auc_scores = []
        
        if multi_class == 'ovr':
            for i in range(num_classes):
                fpr, tpr, _ = roc_curve(y_true_one_hot[:, i], y_scores[:, i])
                auc_score = auc(fpr, tpr)
                auc_scores.append(auc_score)
            return np.mean(auc_scores)
        
        elif multi_class == 'ovo':
            auc_sum = 0
            n_pairs = 0
            for i in range(num_classes):
                for j in range(i+1, num_classes):
                    mask = np.logical_or(y_true == i, y_true == j)
                    y_true_binary = (y_true[mask] == i).astype(int)
                    y_scores_binary = y_scores[mask, i] / (y_scores[mask, i] + y_scores[mask, j])
                    fpr, tpr, _ = roc_curve(y_true_binary, y_scores_binary)
                    auc_sum += auc(fpr, tpr)
                    n_pairs += 1
            return auc_sum / n_pairs if n_pairs > 0 else 0
        
        return auc_scores

    def confusion_matrix(self, y_true, y_pred, plot=True, save_path=None):
        """
        Вычисление и визуализация confusion matrix с возможностью сохранения
        
        Args:
            y_true: истинные метки (batch_size,)
            y_pred: предсказанные метки (batch_size,)
            plot: флаг для отображения визуализации
            save_path: путь для сохранения графика (например, 'confusion_matrix.png')
            
        Returns:
            confusion matrix
        """
        y_true = torch.tensor(y_true, dtype=torch.long).to(self.device)
        y_pred = torch.tensor(y_pred, dtype=torch.long).to(self.device)
        
        num_classes = len(torch.unique(y_true))
        cm = torch.zeros((num_classes, num_classes), dtype=torch.long)
        
        for t, p in zip(y_true, y_pred):
            cm[t, p] += 1
        
        if plot:
            plt.figure(figsize=(10, 8))
            sns.heatmap(cm.numpy(), annot=True, fmt='d', cmap='Blues')
            plt.title('Confusion Matrix')
            plt.ylabel('True Label')
            plt.xlabel('Predicted Label')
            
            if save_path:
                # Создаём директорию, если она не существует
                os.makedirs(os.path.dirname(save_path), exist_ok=True)
                plt.savefig(save_path, dpi=300, bbox_inches='tight')
                print(f"Confusion matrix сохранена в {save_path}")
            
            plt.show()
            plt.close()
        
        return cm



from torch.utils.data import Dataset
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import numpy as np
import torch

class CustomDataset(Dataset):
    def __init__(self, path_file, numeric_columns: list, string_columns: list, binary_columns: list, target_column: str):
        self.path_file = path_file
        self.numeric_columns = numeric_columns
        self.string_columns = string_columns
        self.binary_columns = binary_columns
        self.target_column = target_column
        self.label_encoders = {}
        self.one_hot_encoders = {}

        self.df = pd.read_csv(path_file)
        self.df = self.df.dropna()

        self._convert_numeric()
        self._convert_string()
        self._convert_binary()

    def _convert_numeric(self):
        if self.numeric_columns:
            scaler = StandardScaler()
            self.df[self.numeric_columns] = scaler.fit_transform(self.df[self.numeric_columns])

    def _convert_string(self):
        if self.string_columns:
            for column in self.string_columns:
                le = LabelEncoder()
                self.df[column] = self.df[column].fillna('missing')
                self.df[column] = le.fit_transform(self.df[column])
                self.label_encoders[column] = le

    def _convert_binary(self):
        if self.binary_columns:
            for column in self.binary_columns:
                encoder = OneHotEncoder(sparse_output=False, drop='first')
                transformed = encoder.fit_transform(self.df[[column]])
                new_columns = [f"{column}_{cat}" for cat in encoder.categories_[0][1:]]
                self.df[new_columns] = transformed
                self.df = self.df.drop(column, axis=1)
                self.one_hot_encoders[column] = encoder

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        data_one_row = self.df.iloc[index]
        train = data_one_row.drop(self.target_column).values.astype(np.float32)  # Преобразуем в numpy float32
        target = np.array(data_one_row[self.target_column], dtype=np.float32)  # Преобразуем в numpy float32
        return torch.tensor(train), torch.tensor(target)  # Возвращаем тензоры

class LinearModel(nn.Module):
    def __init__(self,in_features,out_features):
        super().__init__()
        self.layer1 = nn.Linear(in_features,out_features)
    
    def forward(self,x):
        return self.layer1(x)

<h3>Описание названия колонок:</h3>
<ul>
<li>survival - Выживание (0 = Нет; 1 = Да)(target)</li>
<li>pclass - Класс пассажира (1 = 1-й; 2 = 2-й; 3 = 3-й)</li>
<li>sex - Пол</li>
<li>age - Возраст</li>
<li>sibsp - Количество братьев/сестер или супругов на борту</li>
<li>parch - Количество родителей/детей на борту</li>
<li>fare - Стоимость билета пассажира</li>
<li>embarked - Порт посадки (C = Шербур; Q = Квинстаун; S = Саутгемптон)</li>
<li>Age_priority - приоритет для детей 0-10 лет </li>
<li>Has_1_to_3_Parch - семья от 1 до 3 человек </li>
<li>Is_First_Class - человек живет в первом классе</li>
</ul>

1.1 Сравнение моделей разной глубины (15 баллов)
 <br>Создайте и обучите модели с различным количеством слоев:<br>
 - 1 слой (линейный классификатор)
 - 2 слоя (1 скрытый)
 - 3 слоя (2 скрытых)
 - 5 слоев (4 скрытых)
 - 7 слоев (6 скрытых)
 
 Для каждого варианта:
 - Сравните точность на train и test
 - Визуализируйте кривые обучения
 - Проанализируйте время обучения

Создадим отдельные модели через nn.Sequential


In [26]:
model_layer_1 = nn.Sequential(nn.Linear(11,1))
model_layer_2 = nn.Sequential(nn.Linear(11,8),nn.Linear(8,1))
model_layer_3 = nn.Sequential(nn.Linear(11,8),nn.Linear(8,6),nn.Linear(6,1))
model_layer_5 = nn.Sequential(nn.Linear(11,8),nn.Linear(8,6),nn.Linear(6,4),nn.Linear(4,2),nn.Linear(2,1))
model_layer_7 = nn.Sequential(nn.Linear(11,8),nn.Linear(8,6),nn.Linear(6,4),nn.Linear(4,4),nn.Linear(4,4),nn.Linear(4,2),nn.Linear(2,1))

all_model =[model_layer_1,model_layer_2,model_layer_3,model_layer_5,model_layer_7]


7

In [2]:
import torch
#from lesson_2 import CustomDataset

In [46]:
from torch.utils.data import DataLoader,SubsetRandomSampler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt  



def train_model(dataset,model):
    batch_size: int = 32,
    num_epochs: int = 100,
    learning_rate = 0.001,
    l1_lambda: float = 0.01,
    patience: int = 5,
    validation_split: float = 0.2
    lr = 0.001

    in_features = len(dataset.df.columns) - 1  # Все колонки, кроме целевой
    
    out_features = 1 

    #model_layer_1 = nn.Sequential(nn.Linear(in_features,1))
    model = model
    criterion = nn.BCEWithLogitsLoss()  # Функция потерь для регрессии
    optimizer = torch.optim.Adam(params =model.parameters(), lr=lr)

    # Разделяем данные на обучающую и валидационную выборки
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    train_indices, val_indices = train_test_split(
        indices, test_size=validation_split, random_state=42
    )

    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=32, sampler=val_sampler)

    # Инициализируем раннюю остановку


    # Обучение
    print("Starting training...")
    arr_val_loss = []
    arr_train_loss = []
    for epoch in range(60):
        # Обучающий режим
        model.train()
        train_loss = 0.0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to( dtype=torch.float32), target.to(dtype=torch.float32)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target.view(-1, 1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        # Валидация
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to( dtype=torch.float32), target.to( dtype=torch.float32)
                output = model(data)
                loss = criterion(output, target.view(-1, 1))  # Только MSE для валидации
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        arr_val_loss.append(avg_val_loss)
        arr_train_loss.append(avg_train_loss)
        if epoch%20==0:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")
    plt.plot(arr_val_loss)
    plt.plot(arr_train_loss)
    plt.title(f"model_layer_{len(model)}")
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.legend(['Val', 'Train'], loc='upper left')  
    plt.savefig(f'plots/plots_without_accurasy/model_layer_{len(model)}.png')
    plt.close()

In [47]:
dataset = CustomDataset(path_file="data/titanic_new.csv",numeric_columns=["Age","Fare"],string_columns=[],binary_columns=[]
                       
                        ,target_column="Survived")

for model in all_model:
    print(f"Now train model_layer_{len(model)}")
    train_model(dataset,model)
    print("Succes!")
    print()

Now train model_layer_1
Starting training...
Epoch 1/(100,), Train Loss: 0.445789, Val Loss: 0.517851
Epoch 21/(100,), Train Loss: 0.447471, Val Loss: 0.456000
Epoch 41/(100,), Train Loss: 0.446935, Val Loss: 0.463320
Succes!

Now train model_layer_2
Starting training...
Epoch 1/(100,), Train Loss: 0.445267, Val Loss: 0.553689
Epoch 21/(100,), Train Loss: 0.487153, Val Loss: 0.469601
Epoch 41/(100,), Train Loss: 0.461984, Val Loss: 0.464636
Succes!

Now train model_layer_3
Starting training...
Epoch 1/(100,), Train Loss: 0.467646, Val Loss: 0.447486
Epoch 21/(100,), Train Loss: 0.461505, Val Loss: 0.463796
Epoch 41/(100,), Train Loss: 0.461859, Val Loss: 0.484375
Succes!

Now train model_layer_5
Starting training...
Epoch 1/(100,), Train Loss: 0.457384, Val Loss: 0.489109
Epoch 21/(100,), Train Loss: 0.432891, Val Loss: 0.490020
Epoch 41/(100,), Train Loss: 0.445492, Val Loss: 0.479709
Succes!

Now train model_layer_7
Starting training...
Epoch 1/(100,), Train Loss: 0.498698, Val Loss:

Смотря на графики мы видим что разница между количеством слоёв в результате небольшая,также видим не стабильное изменение loss  в разных эпохах,хотя ,например,разница по времени обучения между 1 слоём и 7 существенна

# 1.2 Анализ переобучения (15 баллов)

 Исследуйте влияние глубины на переобучение:
 - Постройте графики train/test accuracy по эпохам
 - Определите оптимальную глубину для каждого датасета
 - Добавьте Dropout и BatchNorm, сравните результаты
 - Проанализируйте, когда начинается переобучение

In [58]:
from torch.utils.data import DataLoader,SubsetRandomSampler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt  



def train_model_with_accurasy(dataset,model):
    batch_size: int = 32,
    num_epochs: int = 100,
    learning_rate = 0.001,
    l1_lambda: float = 0.01,
    patience: int = 5,
    validation_split: float = 0.2
    lr = 0.001

    in_features = len(dataset.df.columns) - 1  # Все колонки, кроме целевой
    
    out_features = 1 

    #model_layer_1 = nn.Sequential(nn.Linear(in_features,1))
    model = model
    criterion = nn.BCEWithLogitsLoss()  # Функция потерь для регрессии
    optimizer = torch.optim.Adam(params =model.parameters(), lr=lr)

    # Разделяем данные на обучающую и валидационную выборки
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    train_indices, val_indices = train_test_split(
        indices, test_size=validation_split, random_state=42
    )

    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=32, sampler=val_sampler)

    # Инициализируем раннюю остановку


    # Обучение
    print("Starting training...")
    arr_val_accurasy = []
    arr_train_accurasy = []
    for epoch in range(60):
        # Обучающий режим
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to( dtype=torch.float32), target.to(dtype=torch.float32)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target.view(-1, 1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            # Вычисляем точность для тренировочной выборки
            preds = (torch.sigmoid(output) > 0.5).float()  # Преобразуем логиты в предсказания (0 или 1)
            train_correct += (preds == target.view(-1, 1)).sum().item()
            train_total += target.size(0)
            
        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = train_correct / train_total
        
        # Валидация
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to( dtype=torch.float32), target.to( dtype=torch.float32)
                output = model(data)
                loss = criterion(output, target.view(-1, 1))  # Только MSE для валидации
                val_loss += loss.item()

                # Вычисляем точность для валидационной выборки
                preds = (torch.sigmoid(output) > 0.5).float()  # Преобразуем логиты в предсказания
                val_correct += (preds == target.view(-1, 1)).sum().item()
                val_total += target.size(0)

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = val_correct / val_total
        arr_val_accurasy.append(val_accuracy)
        arr_train_accurasy.append(train_accuracy)
        
        if epoch%20==0:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")
    plt.plot(arr_val_accurasy)
    plt.plot(arr_train_accurasy)
    plt.title(f"model_layer_5")
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.legend(['Val', 'Train'], loc='upper left')  
    plt.savefig(f'plots/plots_with_accurasy_dropout/model_layer_5.png')
    plt.close()

In [ ]:
dataset = CustomDataset(path_file="data/titanic_new.csv",numeric_columns=["Age","Fare"],string_columns=[],binary_columns=[]
                       
                        ,target_column="Survived")

for model in all_model:
    print(f"Now train model_layer_{len(model)}")
    train_model_with_accurasy(dataset,model)
    print("Succes!")
    print()

Now train model_layer_1
Starting training...
Epoch 1/(100,), Train Loss: 0.435743, Val Loss: 0.475915
Epoch 21/(100,), Train Loss: 0.438628, Val Loss: 0.454661
Epoch 41/(100,), Train Loss: 0.442064, Val Loss: 0.500347
Succes!

Now train model_layer_2
Starting training...
Epoch 1/(100,), Train Loss: 0.465860, Val Loss: 0.507726
Epoch 21/(100,), Train Loss: 0.434508, Val Loss: 0.448856
Epoch 41/(100,), Train Loss: 0.453488, Val Loss: 0.472664
Succes!

Now train model_layer_3
Starting training...
Epoch 1/(100,), Train Loss: 0.470745, Val Loss: 0.485388
Epoch 21/(100,), Train Loss: 0.441853, Val Loss: 0.478778
Epoch 41/(100,), Train Loss: 0.436443, Val Loss: 0.479067
Succes!

Now train model_layer_5
Starting training...
Epoch 1/(100,), Train Loss: 0.463183, Val Loss: 0.445459
Epoch 21/(100,), Train Loss: 0.454532, Val Loss: 0.525905
Epoch 41/(100,), Train Loss: 0.456850, Val Loss: 0.523833
Succes!

Now train model_layer_7
Starting training...
Epoch 1/(100,), Train Loss: 0.501815, Val Loss:

В целом разница по слоям незначительна,но всё же результаты в среднем лучше на пяти слоях

Сейчас добавлю в эту модель Dropout и Batchnorm

In [59]:
model_layer_5_ = nn.Sequential(nn.Linear(11,8),nn.Dropout(0.2),nn.BatchNorm1d(8),
                              nn.Linear(8,6),nn.Dropout(0.2),nn.BatchNorm1d(6),
                              nn.Linear(6,4),nn.Dropout(0.2),nn.BatchNorm1d(4),
                              nn.Linear(4,2),nn.Dropout(0.2),nn.BatchNorm1d(2),
                              nn.Linear(2,1))

In [60]:
train_model_with_accurasy(dataset,model)

Starting training...
Epoch 1/(100,), Train Loss: 0.441318, Val Loss: 0.473886
Epoch 21/(100,), Train Loss: 0.456014, Val Loss: 0.516599
Epoch 41/(100,), Train Loss: 0.475269, Val Loss: 0.514133


График стал более гладким,переобучение пошло примерно с тридцатой эпохи